In [1]:
import sys
sys.path.append('..')
import processing
import configs.common as cc

loader = processing.DatasetLoader('E:\\GitHub\\dataset\\np_dataset')
train_dataloader, test_dataloader = loader.get_dataloaders()
# random_sample = loader.get_random_sample('train')
# random_sample
for src, trg, meta in train_dataloader:
    break
# src, trg, meta = random_sample

In [2]:
from train import new_model
import torch
type = 'transformer'
model_name = 'loss_0.10_time_2025-04-23-08-40-46.pth'
model = new_model(type)
model.load_state_dict(torch.load(f'../pretrained/{type}/{model_name}'))
model.to('cuda')

Transformer(
  (token_embedding_table): Embedding(963, 256)
  (metadata_embedding_table): Embedding(154, 256)
  (positional_encoding): PositionalEncoding()
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-7): 8 x Head(
            (key): Linear(in_features=256, out_features=32, bias=False)
            (query): Linear(in_features=256, out_features=32, bias=False)
            (value): Linear(in_features=256, out_features=32, bias=False)
            (dropout): Dropout(p=0.01, inplace=False)
          )
        )
        (proj): Linear(in_features=256, out_features=256, bias=True)
        (dropout): Dropout(p=0.01, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=256, out_features=1024, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1024, out_features=256, bias=True)
          (3): Dropout(p=0.01, inplace=False)
        )
      )
      (ln1): Lay

In [ ]:
def get_weight(input):
    wegihts = torch.zeros_like(input)
    if prev_tok < cc.start_idx['dyn']:
        
        pitch = prev_tok - cc.start_idx['pitch']
    elif cc.start_idx['dyn'] <= prev_tok and prev_tok < cc.start_idx['length']:
        dynamic = prev_tok - cc.start_idx['dyn']
    elif cc.start_idx['length'] <= prev_tok and prev_tok < cc.start_idx['time']:
        length = prev_tok - cc.start_idx['length']
    elif cc.start_idx['time'] <= prev_tok and prev_tok < cc.start_idx['channel']:
        time_delta = prev_tok - cc.start_idx['time']
    elif cc.start_idx['channel'] <= prev_tok and prev_tok < cc.start_idx['tempo']:
        channel = prev_tok - cc.start_idx['channel']
    elif cc.start_idx['tempo'] <= prev_tok:
        tempo = prev_tok - cc.start_idx['tempo']

In [ ]:
def pick_distributions_by_prev_token(
        input_tokens: torch.Tensor,           # [BATCH, SEQUENCE_LEN], LongTensor of token indices
        distributions: torch.Tensor           # [6, VOCAB_SIZE]
    ) -> torch.Tensor:
    """
    For each token in the sequence (except the first), select a VOCAB_SIZE distribution
    from `distributions`, using the previous token (mod 6).

    Args:
        input_tokens: [BATCH, SEQUENCE_LEN] token indices
        distributions: [6, VOCAB_SIZE] possible distributions

    Returns:
        output: [BATCH, SEQUENCE_LEN, VOCAB_SIZE], where the first position is usually dummy (can be zeros)
    """
    BATCH, SEQ_LEN = input_tokens.size()
    device = input_tokens.device
    VOCAB_SIZE = distributions.size(1)

    # Previous token indices for each position in the sequence (shifted right)
    prev_indices = torch.zeros_like(input_tokens)

    # Map prev token index to one of 6 distributions
    dist_indices = torch.remainder(prev_indices, 6)  # [BATCH, SEQ_LEN], values in 0..5

    # Retrieve the per-token distributions
    # We'll use advanced indexing
    # First, flatten so we can gather using a single index
    flat_dist_indices = dist_indices.view(-1)  # [(BATCH * SEQ_LEN),]
    chosen_dists = distributions[flat_dist_indices]  # [(BATCH*SEQ_LEN), VOCAB_SIZE]

    # Reshape back to [BATCH, SEQ_LEN, VOCAB_SIZE]
    output = chosen_dists.view(BATCH, SEQ_LEN, VOCAB_SIZE)
    return output

In [15]:
cc.start_idx

{'pitch': 1,
 'dyn': 129,
 'length': 257,
 'time': 357,
 'channel': 457,
 'tempo': 713}

In [20]:
boundaries = [cc.start_idx['pitch'] - 1,
              cc.start_idx['dyn'] - 1,
              cc.start_idx['length'] - 1,
              cc.start_idx['time'] - 1,
              cc.start_idx['channel'] - 1,
              cc.start_idx['tempo'] - 1]
        
bins = torch.tensor(boundaries, device=trg.device)
    # Each token is assigned to a bucket 0..len(boundaries)
    # For example, with bins=[10,20,30]: 
    #   token <10 → 0, 10<=token<20 → 1, 20<=token<30 → 2, 30<=token → 3
buckets = torch.bucketize(trg, bins, right=False)
buckets[0]

tensor([5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 6, 5, 1, 2, 3, 6, 5, 1, 2, 3, 4, 6, 5, 1,
        2, 3, 4, 6, 5, 1, 2, 3, 6, 5, 1, 2, 3, 6, 5, 1, 2, 3, 6, 5, 1, 2, 3, 4,
        6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 6, 5, 1, 2, 3, 4, 6,
        5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6,
        5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6,
        5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6,
        5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6,
        5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6,
        5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 6, 5,
        1, 2, 3, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1,
        2, 3, 4, 6, 5, 1, 2, 3, 6, 5, 1, 2, 3, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3,
        4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3,
        6, 5, 1, 2, 3, 4, 6, 5, 1, 2, 3,

In [19]:
trg[0]

tensor([457,  71, 256, 265, 357, 961, 457,  59, 256, 265, 961, 457,  63, 256,
        265, 961, 457,  72, 256, 337, 381, 961, 457,  56, 204, 281, 357, 961,
        457,  44, 207, 281, 961, 457,  60, 256, 337, 961, 457,  63, 255, 337,
        961, 457,  44, 197, 273, 393, 961, 457,  51, 197, 273, 357, 961, 457,
         56, 202, 273, 385, 961, 457,  60, 215, 281, 961, 457,  72, 256, 277,
        389, 961, 457,  63, 211, 281, 357, 961, 457,  75, 256, 281, 389, 961,
        457,  68, 205, 281, 357, 961, 457,  72, 221, 277, 393, 961, 457,  80,
        256, 277, 357, 961, 457,  84, 256, 281, 385, 961, 457,  75, 219, 277,
        361, 961, 457,  80, 256, 277, 389, 961, 457,  87, 255, 277, 357, 961,
        457,  84, 255, 277, 389, 961, 457,  92, 256, 277, 357, 961, 457,  96,
        256, 277, 389, 961, 457,  87, 251, 277, 357, 961, 457,  92, 250, 277,
        389, 961, 457,  99, 256, 277, 357, 961, 457, 104, 255, 273, 389, 961,
        457,  96, 255, 277, 357, 961, 457,  99, 243, 277, 389, 9

In [11]:
def generate(model, context_len, token_ids, meta_ids, num_tokens=1000, device='cpu'):
    model.eval()
    generated = token_ids.detach().cpu().numpy().tolist()[0]

    with torch.no_grad():
            for _ in range(num_tokens):
                print(_, end="\r")
                logits = model(token_ids, meta_ids)  # (1, seq_len, vocab_size)
                logits_last = logits[:, -1, :]       # (1, vocab_size)
                probs = torch.softmax(logits_last, dim=-1)  # (1, vocab_size)

                # Get top-5
                topk_probs, topk_indices = torch.topk(probs, 2, dim=-1)  # Each: shape (1, 5)

                # Normalize to sum to 1
                topk_probs = topk_probs / topk_probs.sum(dim=-1, keepdim=True)

                # Sample
                next_token_idx = torch.multinomial(topk_probs, num_samples=1)  # (1, 1), value in [0..4]
                next_token = topk_indices.gather(1, next_token_idx)

                generated.append(next_token.item())

                token_ids = torch.cat([token_ids, next_token], dim=1)
                token_ids = token_ids[:, -context_len:]

            # If you want metadata, sample or set as zeros:
            # e.g., meta_ids = torch.cat([meta_ids, new_meta], dim=1)[:, -context_len:]

    return generated

new_seq = generate(model, cc.config.values.block_len, src[0].unsqueeze(0), meta[0].unsqueeze(0), num_tokens=500, device='cuda')

In [12]:
decoded_notes_old = processing.decode(src[0])
processing.note_to_midi(decoded_notes_old, "comparison.mid")

decoded_notes_new = processing.decode(new_seq)
processing.note_to_midi(decoded_notes_new, "generated.mid")

In [6]:
decoded_notes_new

[MIDI_note(pitch=70, time_start=0.0, time_end=0.06756756756756757, dynamic=95, channel=29, tempo=148.0),
 MIDI_note(pitch=65, time_start=0.0, time_end=0.10135135135135136, dynamic=95, channel=24, tempo=148.0),
 MIDI_note(pitch=61, time_start=0.11824324324324326, time_end=0.4898648648648649, dynamic=86, channel=29, tempo=148.0),
 MIDI_note(pitch=65, time_start=0.13513513513513514, time_end=0.4560810810810811, dynamic=95, channel=29, tempo=148.0),
 MIDI_note(pitch=61, time_start=0.13513513513513514, time_end=0.40540540540540543, dynamic=95, channel=29, tempo=148.0),
 MIDI_note(pitch=70, time_start=0.13513513513513514, time_end=0.3547297297297297, dynamic=95, channel=54, tempo=148.0),
 MIDI_note(pitch=61, time_start=0.4391891891891892, time_end=1.6554054054054055, dynamic=46, channel=35, tempo=148.0),
 MIDI_note(pitch=30, time_start=0.4391891891891892, time_end=1.6554054054054055, dynamic=85, channel=18, tempo=148.0),
 MIDI_note(pitch=78, time_start=0.4391891891891892, time_end=1.65540540